In [1]:
%%writefile requirements.sh
pip install -qq -U langchain tiktoken pypdf  faiss-gpu unstructured openai
pip install -qq -U transformers InstructorEmbedding sentence_transformers
pip install chromadb


Overwriting requirements.sh


In [5]:
!pip show chromadb

Name: chromadb
Version: 0.4.8
Summary: Chroma.
Home-page: 
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: bcrypt, chroma-hnswlib, fastapi, importlib-resources, numpy, onnxruntime, overrides, posthog, pulsar-client, pydantic, pypika, requests, tokenizers, tqdm, typing-extensions, uvicorn
Required-by: 


In [3]:
!chmod +x ./requirements.sh

In [4]:
!bash ./requirements.sh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.8 MB/s eta 0:00:00
  Using cached chromadb-0.4.8-py3-none-any.whl (418 kB)
  Using cached pydantic-1.10.12-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached chroma-hnswlib-0.7.2.tar.gz (31 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyp

In [6]:
!mkdir db
!mkdir models
## Add your model files to the 'models' folder
!mkdir docs

In [7]:
%%writefile download_kaggle.sh
#!/bin/bash

# Move the kaggle.json file to the correct location (~/.kaggle/)
mkdir -p /root/.kaggle
cp /content/kaggle.json /root/.kaggle/
chmod 600 /root/.kaggle/kaggle.json

# Install the kaggle package (if not already installed)
pip install kaggle

# Download the dataset from Kaggle
kaggle datasets download -d hinepo/harry-potter-books-in-pdf-1-7

# Create the data directory (if not already exists)
mkdir -p docs

# Unzip the downloaded dataset
unzip harry-potter-books-in-pdf-1-7.zip -d docs

# Remove the zip file
rm harry-potter-books-in-pdf-1-7.zip


echo "Dataset downloaded and processed successfully."

Writing download_kaggle.sh


In [8]:
!chmod +x ./download_kaggle.sh

In [9]:
!./download_kaggle.sh

 55% 11.0M/19.9M [00:00<00:00, 57.8MB/s]
100% 19.9M/19.9M [00:00<00:00, 89.1MB/s]
Archive:  harry-potter-books-in-pdf-1-7.zip
  inflating: docs/HP books/Harry Potter - Book 1 - The Sorcerers Stone.pdf  
  inflating: docs/HP books/Harry Potter - Book 2 - The Chamber of Secrets.pdf  
  inflating: docs/HP books/Harry Potter - Book 3 - The Prisoner of Azkaban.pdf  
  inflating: docs/HP books/Harry Potter - Book 4 - The Goblet of Fire.pdf  
  inflating: docs/HP books/Harry Potter - Book 5 - The Order of the Phoenix.pdf  
  inflating: docs/HP books/Harry Potter - Book 6 - The Half-Blood Prince.pdf  
  inflating: docs/HP books/Harry Potter - Book 7 - The Deathly Hallows.pdf  
Dataset downloaded and processed successfully.


In [10]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, PDFMinerLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma, FAISS
import os


persist_directory = "db"

def main():
    for root, dirs, files in os.walk("docs"):
        for file in files:
            if file.endswith(".pdf"):
                print(file)
                loader = PyPDFLoader(os.path.join(root, file))
    documents = loader.load()
    print("splitting into chunks")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    texts = text_splitter.split_documents(documents)
    #create embeddings here
    print("Loading sentence transformers model")
    embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    #create vector store here
    print(f"Creating embeddings. May take some minutes...")
    db = Chroma.from_documents(texts, embeddings, persist_directory=persist_directory)
    db.persist()
    db=None

    print(f"Ingestion complete! You can now run privateGPT.py to query your documents")

if __name__ == "__main__":
    main()

Harry Potter - Book 5 - The Order of the Phoenix.pdf
Harry Potter - Book 2 - The Chamber of Secrets.pdf
Harry Potter - Book 3 - The Prisoner of Azkaban.pdf
Harry Potter - Book 1 - The Sorcerers Stone.pdf
Harry Potter - Book 4 - The Goblet of Fire.pdf
Harry Potter - Book 7 - The Deathly Hallows.pdf
Harry Potter - Book 6 - The Half-Blood Prince.pdf
splitting into chunks
Loading sentence transformers model


Creating embeddings. May take some minutes...
Ingestion complete! You can now run privateGPT.py to query your documents


In [11]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.6/188.6 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 32.9 MB/s eta 0:00:00


In [12]:
!streamlit run chatbot_app.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.83.171.0:8501

  Stopping...
  Stopping...
